In [1]:
import scipy 
import glob
import numpy as np

In [24]:
skeleton_file_path = glob.glob('data/UTD_MAAD/Skeleton/*.mat')
inertia_file_path = glob.glob('data/UTD_MAAD/Inertial/*.mat')

In [43]:
a = skeleton_file_path[2].split('/')[-1]

In [13]:
b = a.split('_')
c = ('').join(b[:-1])

In [50]:
file_map = {}
for i in range(len(inertia_file_path)):
    identity = inertia_file_path[i].split('/')[-1].split('_')[:-1]
    identity = ('').join(identity)
    file_map[identity] = i


In [51]:
a = skeleton_file_path[2].split('/')[-1]
b = a.split('_')
c = ('').join(b[:-1])

In [53]:
file_map[c]

115

In [23]:
a = kf.split(x)

In [25]:
 next_ind = next(a)

In [31]:
train_index, test_index = next(a)

In [32]:
train_index.shape

(60,)

In [3]:
import torch
import numpy as np
import torch.nn.functional as F
from Make_Dataset import Poses3d_Dataset, Utd_Dataset
import torch.nn as nn
import PreProcessing_ncrc
from Models.model_crossview_fusion import ActTransformerMM
from Models.model_acc_only import ActTransformerAcc
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True



# Parameters
print("Creating params....")
params = {'batch_size':8,
          'shuffle': True,
          'num_workers': 0}

num_epochs = 250

# Generators
#pose2id,labels,partition = PreProcessing_ncrc_losocv.preprocess_losocv(8)


dataset = 'utd'
mocap_frames = 100
acc_frames = 150
num_joints = 20
num_classes = 27

if dataset == 'ncrc':
    tr_pose2id,tr_labels,valid_pose2id,valid_labels,pose2id,labels,partition = PreProcessing_ncrc.preprocess()
    training_set = Poses3d_Dataset( data='ncrc',list_IDs=partition['train'], labels=tr_labels, pose2id=tr_pose2id, mocap_frames=mocap_frames, acc_frames=acc_frames, normalize=False)
    training_generator = torch.utils.data.DataLoader(training_set, **params) #Each produced sample is  200 x 59 x 3

    validation_set = Poses3d_Dataset(data='ncrc',list_IDs=partition['valid'], labels=valid_labels, pose2id=valid_pose2id, mocap_frames=mocap_frames, acc_frames=acc_frames ,normalize=False)
    validation_generator = torch.utils.data.DataLoader(validation_set, **params) #Each produced sample is 6000 x 229 x 3

else:

    test_set = Utd_Dataset('/home/bgu9/Fall_Detection_KD_Multimodal/data/UTD_MAAD/valid_data.npz')
    test_generator = torch.utils.data.DataLoader(test_set, **params)


#
#Define model
print("Initiating Model...")

student_model = ActTransformerAcc(device = device, acc_frames=150, num_joints=num_joints, in_chans=3, acc_coords=3,
                                  acc_features=18, has_features = False,num_classes=num_classes)
student_model.load_state_dict(torch.load('/home/bgu9/Fall_Detection_KD_Multimodal/exps/utd/utd_utd_ckpt_wdistance.pt'))

student_model.cuda()

student_model.eval()
y_true = []
y_pred = []
val_loss = 0
val_accuracy = 0
val_t_accuracy = 0
cnt = 0.
student_model=student_model.to(device)
with torch.no_grad():
    for inputs,targets in test_generator:
        y_true.extend(targets.numpy().tolist())
        inputs = inputs.to(device); #print("Validation input: ",inputs)
        targets = targets.to(device)
        
        
        out, student_logits,predictions = student_model(inputs.float())
        loss_score = F.cross_entropy(predictions, targets)
        y_pred.extend(torch.argmax(predictions, 1).cpu().numpy().tolist())
        with torch.no_grad():
            val_loss += loss_score.sum().item()
            val_accuracy += (torch.argmax(predictions, 1) == targets).sum().item()

        cnt += len(targets)
    val_loss /= cnt
    val_accuracy *= 100. / cnt


# compute the confusion matrix
print(y_true)
print(y_pred)
cm = confusion_matrix(y_true, y_pred)

# plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
plt.xticks(np.arange(27))
plt.yticks(np.arange(27))
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix")
plt.show()
print(f"Val accuracy:  {val_accuracy:6.2f} %, Val loss:  {val_loss:8.5f}% , Val_teach: {val_t_accuracy:8.5f}%")

True
Creating params....
Initiating Model...
ACCELERATION Only Model!
-------------ACCELERATION-------------
Acc Frames:  150
Acc embed dim:  32
Acc depth:  4
-------------Regularization-----------
Drop Rate:  0.0
Attn drop rate:  0.0
Drop path rate:  0.2
[24, 11, 18, 16, 25, 15, 3, 13, 23, 22, 1, 11, 21, 12, 4, 5, 11, 23, 13, 12, 4, 26, 17, 7, 14, 3, 5, 21, 0, 3, 22, 23, 18, 18, 24, 10, 21, 24, 20, 9, 20, 24, 12, 23, 20, 18, 5, 25, 7, 19, 13, 9, 7, 2, 14, 17, 7, 10, 16, 22, 25, 2, 13, 19, 21, 15, 2, 6, 8, 14, 17, 16, 2, 17, 10, 1, 26, 14, 19, 22, 15, 0, 8, 1, 25, 4, 26, 0, 8, 6, 10, 9, 11, 20, 4, 16, 8, 5, 12, 6, 1, 9, 26, 0, 6, 15, 19, 3]
[24, 11, 15, 6, 25, 15, 3, 15, 26, 22, 8, 11, 13, 12, 18, 5, 11, 26, 13, 12, 3, 22, 17, 10, 15, 3, 5, 21, 8, 3, 25, 23, 3, 15, 26, 9, 13, 24, 6, 9, 4, 24, 12, 26, 6, 15, 5, 25, 10, 6, 13, 10, 2, 2, 15, 17, 2, 10, 6, 25, 25, 15, 13, 6, 21, 15, 2, 13, 10, 5, 17, 6, 2, 17, 10, 8, 22, 14, 6, 22, 15, 8, 10, 8, 25, 4, 22, 8, 8, 6, 10, 9, 11, 6, 4, 4, 10, 

Val accuracy:   50.00 %, Val loss:   0.19934% , Val_teach:  0.00000%
